# CITIES IN EUROPE

by MG

## 1 - Dataframe with Living Cost

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("all imported")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [4]:
# Set & Connect to the URL with Data
url = 'https://www.numbeo.com/cost-of-living/region_rankings_current.jsp?region=150'
response = requests.get(url)
response

<Response [200]>

In [5]:
# Parse HTML and save to BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
# Heading + DataFrame
column_names = ['Rank','City','Cost of Living Index','Rent Index','Cost + Rent', 'Groceries', 'Restaurante', 'Local' ]
df_cost = pd.DataFrame(columns = column_names)
df_cost

,Rank,City,Cost of Living Index,Rent Index,Cost + Rent,Groceries,Restaurante,Local


In [7]:
# Rows in Dataframe
table_web = soup.find("table", attrs={"id": "t2"})
      
for tr in table_web.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==8:
        df_cost.loc[len(df_cost)] = row_data

df_cost.head()

,Rank,City,Cost of Living Index,Rent Index,Cost + Rent,Groceries,Restaurante,Local
0,,"Zurich, Switzerland",127.69,63.45,96.85,129.97,114.14,123.46
1,,"Basel, Switzerland",125.55,45.65,87.20,123.86,120.29,110.83
2,,"Geneva, Switzerland",121.19,64.92,94.18,122.39,109.56,110.45
3,,"Lausanne, Switzerland",121.03,51.36,87.59,124.57,109.00,108.73
4,,"Bern, Switzerland",114.25,41.19,79.18,110.21,99.63,130.21


In [8]:
df_cost.shape

(219, 8)

In [9]:
# Split City & Country into 2 columns
df_cost[['City','Country']] = df_cost.City.str.split(", ",expand=True)
df_cost.head()

,Rank,City,Cost of Living Index,Rent Index,Cost + Rent,Groceries,Restaurante,Local,Country
0,,Zurich,127.69,63.45,96.85,129.97,114.14,123.46,Switzerland
1,,Basel,125.55,45.65,87.20,123.86,120.29,110.83,Switzerland
2,,Geneva,121.19,64.92,94.18,122.39,109.56,110.45,Switzerland
3,,Lausanne,121.03,51.36,87.59,124.57,109.00,108.73,Switzerland
4,,Bern,114.25,41.19,79.18,110.21,99.63,130.21,Switzerland


In [10]:
# New DF with columns we need
df_costCity=df_cost[['City','Country','Cost of Living Index']]
df_costCity.head()

,City,Country,Cost of Living Index
0,Zurich,Switzerland,127.69
1,Basel,Switzerland,125.55
2,Geneva,Switzerland,121.19
3,Lausanne,Switzerland,121.03
4,Bern,Switzerland,114.25


## 2 - Dataframe with Coordinates

In [11]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

body = client_d844e3e4c650411ebedb1d6fa6837987.get_object(Bucket='capstonecoursera-donotdelete-pr-yazfuluhrqgz86',Key='worldcities1.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_coordinates = pd.read_excel(body)
df_coordinates.head()

,city,lat,lng,country,iso2,iso3,admin_name,capital,population,id,Unnamed: 10,Unnamed: 11
0,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tokyo,primary,35676000,1392685764,NaN,NaN
1,New York,40.6943,-73.9249,United States,US,USA,New York,,19354922.0,1840034016,NaN,NaN
2,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000,1484247881,NaN,NaN
3,Mumbai,19.0170,72.8570,India,IN,IND,Maharashtra,admin,18978000,1356226629,NaN,NaN
4,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000,1076532519,NaN,NaN


In [13]:
# Rename columns
df_coordinates.rename(columns={' city':'City',' lat ':'Latitude',' lng ':'Longitude',' country ':'Country'}, inplace=True)
df_coordinates.columns

Index(['City', 'Latitude', 'Longitude', 'Country', ' iso2 ', ' iso3 ',
       ' admin_name ', ' capital ', ' population ', ' id ', 'Unnamed: 10',
       'Unnamed: 11'],
      dtype='object')

In [14]:
df_coordinates.head()

,City,Latitude,Longitude,Country,iso2,iso3,admin_name,capital,population,id,Unnamed: 10,Unnamed: 11
0,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tokyo,primary,35676000,1392685764,NaN,NaN
1,New York,40.6943,-73.9249,United States,US,USA,New York,,19354922.0,1840034016,NaN,NaN
2,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000,1484247881,NaN,NaN
3,Mumbai,19.0170,72.8570,India,IN,IND,Maharashtra,admin,18978000,1356226629,NaN,NaN
4,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000,1076532519,NaN,NaN


In [15]:
df_coord=df_coordinates[['City','Latitude','Longitude']]
df_coord.head()

,City,Latitude,Longitude
0,Tokyo,35.6850,139.7514
1,New York,40.6943,-73.9249
2,Mexico City,19.4424,-99.1310
3,Mumbai,19.0170,72.8570
4,Sao Paulo,-23.5587,-46.6250


In [ ]:
#df_coord.columns = df_coord.columns.str.strip()
df_coord['City'] = df_coord['City'].str.strip()

In [54]:
df_coord['City'].unique()

array(['Tokyo', 'New York', 'Mexico City', ..., 'Ambarchik', 'Nordvik',
       'Ennadai'], dtype=object)

## 3 - Final Dataframe


In [55]:
df = pd.merge(df_costCity, df_coord, on='City')
df.head()

,City,Country,Cost of Living Index,Latitude,Longitude
0,Zurich,Switzerland,127.69,47.3800,8.5500
1,Basel,Switzerland,125.55,47.5804,7.5900
2,Geneva,Switzerland,121.19,46.2100,6.1400
3,Geneva,Switzerland,121.19,42.8644,-76.9827
4,Geneva,Switzerland,121.19,41.8833,-88.3242


In [56]:
df.shape

(332, 5)

In [57]:
df.drop_duplicates(subset ="City", keep = 'first', inplace = True)
df['City'].value_counts()

York                          1
Dresden                       1
Exeter                        1
Kharkiv                       1
Dublin                        1
Brno                          1
Skopje                        1
Bilbao                        1
Trieste                       1
Las Palmas de Gran Canaria    1
Derby                         1
Leicester                     1
Edinburgh                     1
Szczecin                      1
Szeged                        1
Montpellier                   1
Constanta                     1
Krasnoyarsk                   1
Warsaw                        1
Lublin                        1
Limerick                      1
Valencia                      1
Newcastle upon Tyne           1
Klaipeda                      1
Zaporizhzhya                  1
Krasnodar                     1
Katowice                      1
Ufa                           1
Budapest                      1
Irkutsk                       1
                             ..
Gent    

In [58]:
df.shape

(194, 5)

In [59]:
df.loc[df['City'] == 'Porto']

,City,Country,Cost of Living Index,Latitude,Longitude
254,Porto,Portugal,48.63,41.15,-8.62


In [72]:
df['Cost of Living Index'] = df['Cost of Living Index'].astype(float)
df.dtypes

City                     object
Country                  object
Cost of Living Index    float64
Latitude                float64
Longitude               float64
dtype: object

In [99]:
df['Country'].unique()


array(['Switzerland', 'Norway', 'Iceland', 'France', 'Denmark', 'Croatia',
       'Luxembourg', 'Netherlands', 'Ireland', 'United Kingdom', 'Italy',
       'Sweden', 'Finland', 'Austria', 'Germany', 'Belgium', 'Malta',
       'Spain', 'Greece', 'Slovenia', 'Estonia', 'Portugal', 'Latvia',
       'Slovakia', 'Lithuania', 'Czech Republic', 'Hungary', 'Russia',
       'Poland', 'Montenegro', 'Bulgaria', 'Bosnia And Herzegovina',
       'Serbia', 'Romania', 'Albania', 'North Macedonia', 'Belarus',
       'Moldova', 'Ukraine', 'Kosovo (Disputed Territory)'], dtype=object)

In [265]:
df.head()

,City,Country,Cost of Living Index,Latitude,Longitude
0,Zurich,Switzerland,127.69,47.3800,8.550
1,Basel,Switzerland,125.55,47.5804,7.590
2,Geneva,Switzerland,121.19,46.2100,6.140
6,Lausanne,Switzerland,121.03,46.5304,6.650
7,Bern,Switzerland,114.25,46.9167,7.467


## 4 - Map Cities

In [264]:
# Define map
Latitude = 47.751569
Longitude = 1.675063
europe_map = folium.Map(location=[Latitude, Longitude], zoom_start=4, width=750, height=500)

# add markers to map
for lat, lng, city, country in zip(df['Latitude'], df['Longitude'], df['Country'], df['City']):
    label = '{}, {}'.format(city, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='#722f37',
        fill=True,
        fill_color='#722f37',
        fill_opacity=0.7,
        parse_html=False).add_to(europe_map)  
    
europe_map

## 5 - Venues

In [213]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'ZQLNZTLKFHBCIGVCJOCENBFBSMY4EFRNOEZ3H3CIRPCRBCGB'
CLIENT_SECRET = 'TRWSNHPUXWK4BO0OMPUK5WHE3KFMCTQOMQXFUFBXSBNLFLPP'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZQLNZTLKFHBCIGVCJOCENBFBSMY4EFRNOEZ3H3CIRPCRBCGB
CLIENT_SECRET:TRWSNHPUXWK4BO0OMPUK5WHE3KFMCTQOMQXFUFBXSBNLFLPP


In [214]:
# Top 20 Venues for the First City

df.loc[10, 'City']

city_latitude = df.loc[0, 'Latitude'] #city latitude value
city_longitude = df.loc[0, 'Longitude'] # city longitude value
city_name = df.loc[0, 'City'] # city name
country_name = df.loc[0, 'Country'] # country name

print('Latitude and longitude values of {}, in {}, are {}, {}.'.format(city_name,
                                                                country_name,
                                                               city_latitude, 
                                                               city_longitude))


Latitude and longitude values of Zurich, in Switzerland, are 47.38, 8.55.


In [215]:
LIMIT = 20
radius = 500 

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZQLNZTLKFHBCIGVCJOCENBFBSMY4EFRNOEZ3H3CIRPCRBCGB&client_secret=TRWSNHPUXWK4BO0OMPUK5WHE3KFMCTQOMQXFUFBXSBNLFLPP&v=20180605&ll=47.38,8.55&radius=500&limit=20'

In [202]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ecab3d360ba08001ba5e93a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Oberstrass',
  'headerFullLocation': 'Oberstrass, Zürich',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 47.384500004500005,
    'lng': 8.556633266001214},
   'sw': {'lat': 47.3754999955, 'lng': 8.543366733998788}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b0e7763f964a520c35723e3',
       'name': 'The Alehouse - Palmhof',
       'location': {'address': 'Universitätsstr. 23',
        'lat': 47.37913804950045,
        'lng': 8.548037081392495,
        'labeledLatLngs': [{'label': 'display',
          'lat':

In [216]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [203]:
# Dataframe with Venues

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Alehouse - Palmhof,Gastropub,47.379138,8.548037
1,K55 GmbH,Electronics Store,47.378553,8.548116
2,Hausammann,Bakery,47.383184,8.548202
3,Kleine Freiheit,Snack Place,47.379216,8.544588
4,"focusTerra, ETH Zürich",Science Museum,47.378231,8.547438


In [217]:
# For all Cities:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [221]:
europe_venues = getNearbyVenues(names=df['City'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

print(europe_venues.shape)

Zurich
Basel
Geneva
Lausanne
Bern
Trondheim
Stavanger
Oslo
Reykjavik
Bergen
Paris
Copenhagen
Aalborg
Dubrovnik
Luxembourg
Amsterdam
Odense
Dublin
Utrecht
London
Milan
Cork
Galway
Stockholm
Uppsala
Brighton
Eindhoven
Groningen
Tampere
Linz
Graz
Haarlem
Lyon
Marseille
Munich
Helsinki
Gent
Maastricht
Parma
Trieste
Genoa
Antwerp
Aberdeen
Brussels
Montpellier
Toulouse
Sliema
Malmo
Rome
Limerick
Rotterdam
Bologna
Lille
Hamburg
Bergamo
Reading
Heidelberg
Florence
Frankfurt
Cambridge
York
Bristol
Oxford
Lincoln
Turku
Nuremberg
Mannheim
Bonn
Stuttgart
Turin
Dusseldorf
Cologne
Nottingham
Edinburgh
Vienna
Birmingham
Dresden
Manchester
Bilbao
Plymouth
Exeter
Dundee
Berlin
Southampton
Bremen
Essen
Derby
Leicester
Hanover
Naples
Liverpool
Glasgow
Leipzig
Newcastle upon Tyne
Karlsruhe
Sheffield
Leeds
Barcelona
Madrid
Belfast
Thessaloniki
Athens
Kingston upon Hull
Granada
Catania
Ljubljana
Split
Palermo
Tallinn
Valencia
Braga
Santa Cruz de Tenerife
Malaga
Funchal
Alicante
Maribor
Rijeka
Zagreb
Tartu
L

In [222]:
print('There are {} uniques categories.'.format(len(europe_venues['Venue Category'].unique())))

There are 328 uniques categories.


In [223]:
europe_venues.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Zurich,47.38,8.55,The Alehouse - Palmhof,47.379138,8.548037,Gastropub
1,Zurich,47.38,8.55,K55 GmbH,47.378553,8.548116,Electronics Store
2,Zurich,47.38,8.55,Hausammann,47.383184,8.548202,Bakery
3,Zurich,47.38,8.55,Kleine Freiheit,47.379216,8.544588,Snack Place
4,Zurich,47.38,8.55,"focusTerra, ETH Zürich",47.378231,8.547438,Science Museum


In [224]:
# ANALYZING IT

# one hot encoding
europe_onehot = pd.get_dummies(europe_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
europe_onehot['City'] = europe_venues['City'] 

# move city column to the first column
fixed_columns = [europe_onehot.columns[-1]] + list(europe_onehot.columns[:-1])
europe_onehot = europe_onehot[fixed_columns]

europe_onehot.head()

,City,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,...,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Zurich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Zurich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Zurich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Zurich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Zurich,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [225]:
#Group
europe_grouped = europe_onehot.groupby('City').mean().reset_index()
europe_grouped

# Print each city + top 10 venues
num_top_venues = 10

for ci in europe_grouped['City']:
    print("----"+ci+"----")
    temp = europe_grouped[europe_grouped['City'] == ci].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

    
# Put in a DataFrame

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


----Aalborg----
                   venue  freq
0      Convenience Store   1.0
1      Accessories Store   0.0
2                   Park   0.0
3               Pharmacy   0.0
4              Pet Store   0.0
5               Pet Café   0.0
6    Peruvian Restaurant   0.0
7     Persian Restaurant   0.0
8  Performing Arts Venue   0.0
9       Pedestrian Plaza   0.0


----Aberdeen----
                   venue  freq
0                  Beach   1.0
1      Accessories Store   0.0
2     Photography Studio   0.0
3              Pet Store   0.0
4               Pet Café   0.0
5    Peruvian Restaurant   0.0
6     Persian Restaurant   0.0
7  Performing Arts Venue   0.0
8       Pedestrian Plaza   0.0
9            Pastry Shop   0.0


----Alicante----
                   venue  freq
0     Spanish Restaurant  0.15
1          Deli / Bodega  0.05
2         Ice Cream Shop  0.05
3              Gastropub  0.05
4      French Restaurant  0.05
5              Nightclub  0.05
6  Performing Arts Venue  0.05
7       Tapas Re

9                 Bar  0.05


----Bristol----
            venue  freq
0     Music Venue  0.10
1     Coffee Shop  0.10
2           Plaza  0.10
3             Pub  0.10
4  Sandwich Place  0.10
5            Café  0.10
6      Steakhouse  0.05
7          Bakery  0.05
8   Train Station  0.05
9            Park  0.05


----Brno----
                        venue  freq
0                        Café  0.25
1                      Bistro  0.10
2                     Theater  0.10
3  Modern European Restaurant  0.05
4           Indian Restaurant  0.05
5                 Pizza Place  0.05
6                    Pet Café  0.05
7              Ice Cream Shop  0.05
8         Indie Movie Theater  0.05
9                    Fountain  0.05


----Brussels----
                           venue  freq
0               Greek Restaurant  0.20
1                 Sandwich Place  0.10
2                          Hotel  0.10
3               Sushi Restaurant  0.05
4                 Cosmetics Shop  0.05
5                   Burger

                           venue  freq
0                       Bus Stop  0.50
1                     Food Truck  0.25
2                Thai Restaurant  0.25
3  Paper / Office Supplies Store  0.00
4                       Pet Café  0.00
5            Peruvian Restaurant  0.00
6             Persian Restaurant  0.00
7          Performing Arts Venue  0.00
8               Pedestrian Plaza  0.00
9                    Pastry Shop  0.00


----Essen----
                       venue  freq
0               Cocktail Bar  0.10
1                  Drugstore  0.10
2     Furniture / Home Store  0.05
3        Indie Movie Theater  0.05
4                Pizza Place  0.05
5             Clothing Store  0.05
6                      Plaza  0.05
7  Middle Eastern Restaurant  0.05
8                     Bakery  0.05
9                      Hotel  0.05


----Exeter----
                        venue  freq
0            Business Service  0.17
1                   Gastropub  0.17
2                 Supermarket  0.17
3  Constr

                  venue  freq
0            Restaurant  0.07
1           Coffee Shop  0.07
2              Bus Stop  0.07
3            Steakhouse  0.07
4        Soccer Stadium  0.07
5          Soccer Field  0.07
6           Snack Place  0.07
7  Gym / Fitness Center  0.07
8              Gym Pool  0.07
9         Track Stadium  0.07


----Kaliningrad----
                    venue  freq
0       Electronics Store  0.10
1                Boutique  0.10
2  Furniture / Home Store  0.05
3           Historic Site  0.05
4                  Museum  0.05
5             Pizza Place  0.05
6             Gaming Cafe  0.05
7             Coffee Shop  0.05
8       Mobile Phone Shop  0.05
9             Auto Garage  0.05


----Karlsruhe----
                       venue  freq
0                      Hotel  0.15
1                  Drugstore  0.10
2                        Zoo  0.05
3                      Plaza  0.05
4                       Café  0.05
5                Supermarket  0.05
6                       Park  0

               venue  freq
0              Plaza  0.15
1              Hotel  0.10
2  Polish Restaurant  0.10
3                Pub  0.10
4         Restaurant  0.05
5                Bar  0.05
6          Jazz Club  0.05
7     History Museum  0.05
8        Pizza Place  0.05
9           Creperie  0.05


----Luxembourg----
                 venue  freq
0   Italian Restaurant  0.10
1         Dessert Shop  0.10
2                 Café  0.10
3  Japanese Restaurant  0.05
4                  Bar  0.05
5  Indie Movie Theater  0.05
6                Hotel  0.05
7     Greek Restaurant  0.05
8   Frozen Yogurt Shop  0.05
9               Palace  0.05


----Lyon----
               venue  freq
0                Bar  0.20
1  French Restaurant  0.15
2              Plaza  0.15
3       Cocktail Bar  0.10
4               Café  0.10
5         Public Art  0.05
6        Coffee Shop  0.05
7    Thai Restaurant  0.05
8         Donut Shop  0.05
9       Neighborhood  0.05


----Maastricht----
                    venue  fre

            venue  freq
0      Restaurant  0.10
1    Concert Hall  0.10
2          Bakery  0.05
3       Irish Pub  0.05
4           Hotel  0.05
5  History Museum  0.05
6       Multiplex  0.05
7     Music Venue  0.05
8       Gastropub  0.05
9            Park  0.05


----Olomouc----
                        venue  freq
0  Construction & Landscaping   1.0
1           Accessories Store   0.0
2                        Park   0.0
3                    Pharmacy   0.0
4                   Pet Store   0.0
5                    Pet Café   0.0
6         Peruvian Restaurant   0.0
7          Persian Restaurant   0.0
8       Performing Arts Venue   0.0
9            Pedestrian Plaza   0.0


----Oradea----
                 venue  freq
0                  Zoo   0.2
1                Hotel   0.2
2                  Gym   0.2
3     Basketball Court   0.2
4                  Bar   0.2
5           Public Art   0.0
6                 Park   0.0
7             Pet Café   0.0
8  Peruvian Restaurant   0.0
9   Persian Res

                venue  freq
0          Art Museum  0.20
1              Garden  0.10
2  Photography Studio  0.05
3             Exhibit  0.05
4               Plaza  0.05
5              Hostel  0.05
6                 Bar  0.05
7         Music Venue  0.05
8               Canal  0.05
9         Art Gallery  0.05


----Samara----
                   venue  freq
0           Liquor Store  0.14
1       Sushi Restaurant  0.14
2               Bus Stop  0.14
3   Gym / Fitness Center  0.14
4  Australian Restaurant  0.14
5      Electronics Store  0.14
6          Grocery Store  0.14
7            Pastry Shop  0.00
8               Pet Café  0.00
9    Peruvian Restaurant  0.00


----Santa Cruz de Tenerife----
               venue  freq
0   Tapas Restaurant  0.15
1       Dessert Shop  0.10
2       Camera Store  0.10
3               Café  0.10
4              Hotel  0.05
5  Electronics Store  0.05
6     Sandwich Place  0.05
7              Plaza  0.05
8               Park  0.05
9       Burger Joint  0.05


--

                   venue  freq
0           Optical Shop  0.67
1          Indie Theater  0.33
2      Accessories Store  0.00
3            Pastry Shop  0.00
4               Pharmacy  0.00
5              Pet Store  0.00
6               Pet Café  0.00
7    Peruvian Restaurant  0.00
8     Persian Restaurant  0.00
9  Performing Arts Venue  0.00


----Trieste----
                   venue  freq
0           Soccer Field  0.25
1          Grocery Store  0.25
2           Skating Rink  0.25
3                   Café  0.25
4               Pet Café  0.00
5    Peruvian Restaurant  0.00
6     Persian Restaurant  0.00
7  Performing Arts Venue  0.00
8       Pedestrian Plaza  0.00
9            Pastry Shop  0.00


----Trondheim----
                   venue  freq
0          Grocery Store  0.33
1            Pizza Place  0.33
2               Bus Stop  0.33
3      Accessories Store  0.00
4               Pet Café  0.00
5    Peruvian Restaurant  0.00
6     Persian Restaurant  0.00
7  Performing Arts Venue  0.00
8

In [227]:
# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = europe_grouped['City']

for ind in np.arange(europe_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(europe_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aalborg,Convenience Store,Zoo,Food,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market
1,Aberdeen,Beach,Zoo,Fair,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop
2,Alicante,Spanish Restaurant,Italian Restaurant,Mediterranean Restaurant,Deli / Bodega,Concert Hall,Restaurant,Pizza Place,Performing Arts Venue,Farmers Market,Nightclub
3,Amsterdam,Coffee Shop,Restaurant,Café,Music Store,French Restaurant,Breakfast Spot,Snack Place,Gym,Roof Deck,Train Station
4,Antwerp,Thai Restaurant,Coffee Shop,Cosmetics Shop,Pub,Comic Shop,Friterie,Opera House,Sushi Restaurant,Sandwich Place,Asian Restaurant


## 6 - Cluster Cities

In [298]:
# set number of clusters
kclusters = 5

europe_grouped_clustering = europe_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(europe_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [247]:
# Dataframe with Cities + Top Venues

# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

europe_merged = df

# merge europe_grouped with europa_data to add latitude/longitude for each city
europe_merged = europe_merged.join(city_venues_sorted.set_index('City'), on='City')

europe_merged.head()

,City,Country,Cost of Living Index,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Zurich,Switzerland,127.69,47.3800,8.550,1.0,Italian Restaurant,Hotel,Restaurant,Bakery,Swiss Restaurant,Middle Eastern Restaurant,Snack Place,Electronics Store,Vegetarian / Vegan Restaurant,Gastropub
1,Basel,Switzerland,125.55,47.5804,7.590,1.0,Fast Food Restaurant,Tram Station,Nightclub,Supermarket,Bar,Sandwich Place,Grocery Store,Toy / Game Store,Lounge,Shopping Mall
2,Geneva,Switzerland,121.19,46.2100,6.140,1.0,Bar,Plaza,Coffee Shop,Breakfast Spot,Moroccan Restaurant,Falafel Restaurant,Seafood Restaurant,Chocolate Shop,Trattoria/Osteria,Park
6,Lausanne,Switzerland,121.03,46.5304,6.650,1.0,Bakery,Bus Stop,Supermarket,Farmers Market,Bus Station,Metro Station,Department Store,Forest,Fountain,Food Truck
7,Bern,Switzerland,114.25,46.9167,7.467,1.0,Japanese Restaurant,Electronics Store,Train Station,Supermarket,Zoo,Flea Market,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [248]:
europe_merged['Cluster Labels'] = europe_merged['Cluster Labels'].fillna(5)
europe_merged['Cluster Labels'].unique()

array([1., 0., 2., 3., 5., 4.])

### Map Clusters

In [281]:
# Define map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=4, width=750, height=500)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(europe_merged['Latitude'], europe_merged['Longitude'], europe_merged['City'], europe_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Look for similar cities to Porto

In [250]:
europe_merged.loc[europe_merged['City'] == 'Porto']

,City,Country,Cost of Living Index,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
254,Porto,Portugal,48.63,41.15,-8.62,1.0,Hotel,Burger Joint,Hostel,Beer Bar,Syrian Restaurant,Bar,Bakery,Sushi Restaurant,Café,Mexican Restaurant


In [274]:
#df_final = europe_merged['Cluster Labels'] == '1.0'
df_cluster1 = europe_merged.loc[europe_merged['Cluster Labels'] == 1.0]
df_cluster1.head()

,City,Country,Cost of Living Index,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Zurich,Switzerland,127.69,47.3800,8.550,1.0,Italian Restaurant,Hotel,Restaurant,Bakery,Swiss Restaurant,Middle Eastern Restaurant,Snack Place,Electronics Store,Vegetarian / Vegan Restaurant,Gastropub
1,Basel,Switzerland,125.55,47.5804,7.590,1.0,Fast Food Restaurant,Tram Station,Nightclub,Supermarket,Bar,Sandwich Place,Grocery Store,Toy / Game Store,Lounge,Shopping Mall
2,Geneva,Switzerland,121.19,46.2100,6.140,1.0,Bar,Plaza,Coffee Shop,Breakfast Spot,Moroccan Restaurant,Falafel Restaurant,Seafood Restaurant,Chocolate Shop,Trattoria/Osteria,Park
6,Lausanne,Switzerland,121.03,46.5304,6.650,1.0,Bakery,Bus Stop,Supermarket,Farmers Market,Bus Station,Metro Station,Department Store,Forest,Fountain,Food Truck
7,Bern,Switzerland,114.25,46.9167,7.467,1.0,Japanese Restaurant,Electronics Store,Train Station,Supermarket,Zoo,Flea Market,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [275]:
df_cluster1['Cluster Labels'].unique()

array([1.])

In [277]:
df_cluster1.drop(['Cluster Labels'], axis=1, inplace=True)
df_cluster1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,City,Country,Cost of Living Index,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Zurich,Switzerland,127.69,47.3800,8.550,Italian Restaurant,Hotel,Restaurant,Bakery,Swiss Restaurant,Middle Eastern Restaurant,Snack Place,Electronics Store,Vegetarian / Vegan Restaurant,Gastropub
1,Basel,Switzerland,125.55,47.5804,7.590,Fast Food Restaurant,Tram Station,Nightclub,Supermarket,Bar,Sandwich Place,Grocery Store,Toy / Game Store,Lounge,Shopping Mall
2,Geneva,Switzerland,121.19,46.2100,6.140,Bar,Plaza,Coffee Shop,Breakfast Spot,Moroccan Restaurant,Falafel Restaurant,Seafood Restaurant,Chocolate Shop,Trattoria/Osteria,Park
6,Lausanne,Switzerland,121.03,46.5304,6.650,Bakery,Bus Stop,Supermarket,Farmers Market,Bus Station,Metro Station,Department Store,Forest,Fountain,Food Truck
7,Bern,Switzerland,114.25,46.9167,7.467,Japanese Restaurant,Electronics Store,Train Station,Supermarket,Zoo,Flea Market,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


### Split into Categories

In [270]:
bins = np.linspace(min(df['Cost of Living Index']), max(df['Cost of Living Index']), 6)
bins

array([ 27.32 ,  47.394,  67.468,  87.542, 107.616, 127.69 ])

In [271]:
group_names = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']
df['Living Cost'] = pd.cut(df['Cost of Living Index'], bins, labels = group_names, include_lowest=True)
df.head()

,City,Country,Cost of Living Index,Latitude,Longitude,Living Cost
0,Zurich,Switzerland,127.69,47.3800,8.550,Very High
1,Basel,Switzerland,125.55,47.5804,7.590,Very High
2,Geneva,Switzerland,121.19,46.2100,6.140,Very High
6,Lausanne,Switzerland,121.03,46.5304,6.650,Very High
7,Bern,Switzerland,114.25,46.9167,7.467,Very High


In [273]:
df['Living Cost'].value_counts()

Very Low     70
Low          61
Moderate     53
Very High     5
High          5
Name: Living Cost, dtype: int64

In [ ]:
df_final = df_cluster1.join(city_venues_sorted.set_index('City'), on='City')

In [279]:
df_final = pd.merge(df_cluster1, df[['City','Living Cost']], on='City')
df_final.head()

,City,Country,Cost of Living Index,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Living Cost
0,Zurich,Switzerland,127.69,47.3800,8.550,Italian Restaurant,Hotel,Restaurant,Bakery,Swiss Restaurant,Middle Eastern Restaurant,Snack Place,Electronics Store,Vegetarian / Vegan Restaurant,Gastropub,Very High
1,Basel,Switzerland,125.55,47.5804,7.590,Fast Food Restaurant,Tram Station,Nightclub,Supermarket,Bar,Sandwich Place,Grocery Store,Toy / Game Store,Lounge,Shopping Mall,Very High
2,Geneva,Switzerland,121.19,46.2100,6.140,Bar,Plaza,Coffee Shop,Breakfast Spot,Moroccan Restaurant,Falafel Restaurant,Seafood Restaurant,Chocolate Shop,Trattoria/Osteria,Park,Very High
3,Lausanne,Switzerland,121.03,46.5304,6.650,Bakery,Bus Stop,Supermarket,Farmers Market,Bus Station,Metro Station,Department Store,Forest,Fountain,Food Truck,Very High
4,Bern,Switzerland,114.25,46.9167,7.467,Japanese Restaurant,Electronics Store,Train Station,Supermarket,Zoo,Flea Market,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Very High


In [303]:
df_final.shape

(174, 16)

In [302]:
df_final.groupby(df_final['Living Cost']).count()

,City,Country,Cost of Living Index,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Living Cost,,,,,,,,,,,,,,,
Very Low,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63
Low,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53
Moderate,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48
High,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
Very High,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5


In [315]:
df_final.loc[df_final['Cost of Living Index'] == (df_final['Cost of Living Index'].min())]

,City,Country,Cost of Living Index,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Living Cost
173,Sumy,Ukraine,27.32,50.9243,34.7809,Arcade,Zoo,Food,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Very Low


In [316]:
df_final.loc[df_final['Cost of Living Index'] == (df_final['Cost of Living Index'].max())]

,City,Country,Cost of Living Index,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Living Cost
0,Zurich,Switzerland,127.69,47.38,8.55,Italian Restaurant,Hotel,Restaurant,Bakery,Swiss Restaurant,Middle Eastern Restaurant,Snack Place,Electronics Store,Vegetarian / Vegan Restaurant,Gastropub,Very High


In [296]:
# Define map
final_map = folium.Map(location=[Latitude, Longitude], zoom_start=4, width=750, height=500)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

def costColors (level):
    if level == 'Very Low':
        return 'darkgreen'
    elif level == 'Low':
        return 'blue'
    elif level == 'Moderate':
        return 'yellow'
    elif level == 'High':
        return 'orange'
    else:
        return 'red'

# add markers to the map
markers_colors = []
for lat, lon, poi, cost, level in zip(df_final['Latitude'], df_final['Longitude'], df_final['City'], df_final['Cost of Living Index'], df_final['Living Cost']):
    label = folium.Popup(str(poi) + ' - Living Cost Index: ' + str(cost), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color = costColors(level),
        fill=True,
        fill_color = costColors(level),
        fill_opacity=0.7).add_to(final_map)

    
final_map